# Deimos Router

Deimos is the smallest, ugliest moon of Mars.

Deimos is also the smallest, dumbest way to do routing.

In [1]:
import deimos_router

In [2]:
from deimos_router import Router, register_router, chat
from deimos_router.rules import TaskRule, AutoTaskRule, CodeRule, CodeLanguageRule, NaturalLanguageRule, MessageLengthRule, ConversationContextRule, Rule, Decision

## Basic Usage - Task Based Routing

A Router uses one or more Rules to select a model. The simplest Rule is TaskRule, which maps task names to models.

In [3]:
TaskRule(
    name="task-based-routing",
    triggers={
       'coding': 'openai/gpt-5',
       'creative': 'openai/gpt-4o',
       'simple': 'openai/gpt-5-nano'
    }
)

TaskRule('task-based-routing', {'coding': 'openai/gpt-5', 'creative': 'openai/gpt-4o', 'simple': 'openai/gpt-5-nano'})

In [4]:
Router(
   name="my-first-router",
   rules=["deimos/rules/task-based-routing"],
   default="openai/gpt-4o-mini"
)



Router('my-first-router', rules=['deimos/rules/task-based-routing'], default='openai/gpt-4o-mini')

To use a Router, make a call to `chat.completions.create`, as if it were a call to OpenAI's SDK, and specify the router as the model.

For simple task-based routing, include the task name as an argument.

In [5]:
# Use the router for chat completions
response = chat.completions.create(
   model="deimos/my-first-router",
   messages=[
       {"role": "user", "content": "Write a python function that finds the nth fibonacci number"}
   ],
   task="coding"
)

print(response.choices[0].message.content)

def fib(n: int) -> int:
    """
    Return the nth Fibonacci number with 0-indexing:
      F(0) = 0, F(1) = 1, F(2) = 1, ...
    Uses fast doubling (O(log n) time).
    """
    if not isinstance(n, int) or n < 0:
        raise ValueError("n must be a non-negative integer")

    def _fib(k: int) -> tuple[int, int]:
        if k == 0:
            return (0, 1)
        a, b = _fib(k >> 1)
        c = a * (2 * b - a)
        d = a * a + b * b
        if k & 1:
            return (d, c + d)
        else:
            return (c, d)

    return _fib(n)[0]


Details about the routing decision can be found in the response at `._deimos_metadata`.

In [6]:
response._deimos_metadata

{'router_used': 'my-first-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'explain': [{'rule_type': 'TaskRule',
   'rule_name': 'task-based-routing',
   'rule_trigger': 'coding',
   'decision': 'openai/gpt-5'}]}

## AutoTask Routing

An `AutoTaskRule` is created in the same way as a `TaskRule`, but the task is determined by a call to a small language model.

In [7]:
AutoTaskRule(
    name = "auto-task-rule",
    triggers = {
        "creative writing" : "openai/gpt-4o",
        "writing code" : "openai/gpt-5",
        "informational" : "openai/gpt-5-mini",
        "haiku composition" : "openai/gpt-5-nano"
    },
)


        

AutoTaskRule('auto-task-rule', {'creative writing': 'openai/gpt-4o', 'writing code': 'openai/gpt-5', 'informational': 'openai/gpt-5-mini', 'haiku composition': 'openai/gpt-5-nano'}, default=None, llm_model='openai/gpt-5-nano')

In [9]:
Router(
    name = "auto-router",
    rules = ["deimos/rules/auto-task-rule"],
    default="openai/gpt-4o-mini"
)


Router('auto-router', rules=['deimos/rules/auto-task-rule'], default='openai/gpt-4o-mini')

In [10]:
# Use the router for chat completions
response1 = chat.completions.create(
   model="deimos/auto-router",
   messages=[
       {"role": "user", "content": "Write a python function that finds the nth fibonacci number"}
   ],
   task="coding"
)

print(response1.choices[0].message.content)

def fib(n: int) -> int:
    """
    Return the nth Fibonacci number using fast doubling.
    0-indexed: fib(0)=0, fib(1)=1, fib(2)=1, ...
    Raises ValueError for negative n.
    Runs in O(log n) time.
    """
    if n < 0:
        raise ValueError("n must be non-negative")

    def _fib(k: int) -> tuple[int, int]:
        if k == 0:
            return (0, 1)
        a, b = _fib(k >> 1)  # a = F(k), b = F(k+1)
        c = a * ((b << 1) - a)  # F(2k)
        d = a * a + b * b       # F(2k+1)
        if k & 1:
            return (d, c + d)
        else:
            return (c, d)

    return _fib(n)[0]


In [11]:
response1._deimos_metadata

{'router_used': 'auto-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'explain': [{'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'writing code',
   'decision': 'openai/gpt-5'}]}

In [12]:
haiku = chat.completions.create(
   model="deimos/auto-router",
   messages=[
       {"role": "user", "content": "Write a short japanese poem about mars, and follow the syllable count 5, 7, 5"}
   ],
)

print(haiku.choices[0].message.content)

砂の星
風が火星を
揺らす砂


In [13]:
haiku._deimos_metadata

{'router_used': 'auto-router',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'explain': [{'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'haiku composition',
   'decision': 'openai/gpt-5-nano'}]}

## Code / Not code Routing

The `CodeRule` is a very simple rule that determines whether a prompt contains code and routes based on that.

In [16]:
CodeRule(
    name = "code-or-not-code",
    code = "openai/gpt-5",
    not_code = "openai/gpt-4o"
)

Router(
    name = "code-nocode-router",
    rules = ["deimos/rules/code-or-not-code"]
)



Router('code-nocode-router', rules=['deimos/rules/code-or-not-code'], default='gpt-3.5-turbo')

In [17]:
prompt = """
Debug this:
```
def multiply(x, y):
    return x**y
```
"""


might_be_code_1 = chat.completions.create(
   model="deimos/code-nocode-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(might_be_code_1.choices[0].message.content)

You’re using exponentiation (**) instead of multiplication. Use the * operator.

Corrected code:
```
def multiply(x, y):
    return x * y
```

Example:
- multiply(3, 4) -> 12
- multiply(2, 5) -> 10


In [18]:
might_be_code_1._deimos_metadata

{'router_used': 'code-nocode-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'code_detected',
   'decision': 'openai/gpt-5'}]}

In [19]:
prompt = """
What is the smallest, ugliest moon of Mars?
"""


might_be_code_2 = chat.completions.create(
   model="deimos/code-nocode-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(might_be_code_2.choices[0].message.content)

Mars has two small moons, Phobos and Deimos. Of the two, Phobos is generally considered the smaller and less aesthetically appealing. Phobos is heavily cratered, irregularly shaped, and has a very rugged surface, which contributes to its "ugly" appearance compared to other moons in the solar system. It is the larger of the two Martian moons but still quite small at about 22.4 kilometers in diameter. Its close orbit around Mars and its dark, dusty surface add to its unique and rather stark appearance.


In [20]:
might_be_code_2._deimos_metadata

{'router_used': 'code-nocode-router',
 'selected_model': 'openai/gpt-4o',
 'original_model_field': 'openai/gpt-4o',
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'no_code_detected',
   'decision': 'openai/gpt-4o'}]}

## Code Language Routing

To route based on programming language, use a CodeLanguage rule.

This rule uses regex to classify among several popular languages, and then falls back to a small language model call to determine language.

In [21]:
CodeLanguageRule(
    name = "code-lang-rule",
    language_mappings = {
        "python" : "openai/gpt-3.5-turbo",
        "sql" : "openai/gpt-5-mini",
    }
)

Router(
    name = "code-lang-router",
    rules = ["deimos/rules/code-lang-rule"]
)


Router('code-lang-router', rules=['deimos/rules/code-lang-rule'], default='gpt-3.5-turbo')

In [22]:
prompt = """
What does this function do?
```
def _idk(x):
    if n < 0:
        raise ValueError("n must be non-negative")
    a, b = 0, 1  # (F(m), F(m+1))
    for bit in bin(n)[2:]:
        c = a * (2 * b - a)     # F(2m)
        d = a * a + b * b       # F(2m+1)
        if bit == '0':
            a, b = c, d
        else:
            a, b = d, c + d
    return a
```
"""

what_it_do = chat.completions.create(
   model="deimos/code-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(what_it_do.choices[0].message.content)

This function calculates the nth Fibonacci number using the iterative method where the Fibonacci numbers are generated by repeatedly adding the two previous numbers in the sequence. It uses bitwise operations to efficiently calculate the Fibonacci number at position n.


In [23]:
what_it_do._deimos_metadata

{'router_used': 'code-lang-router',
 'selected_model': 'openai/gpt-3.5-turbo',
 'original_model_field': 'openai/gpt-3.5-turbo',
 'explain': [{'rule_type': 'CodeLanguageRule',
   'rule_name': 'code-lang-rule',
   'rule_trigger': 'python',
   'decision': 'openai/gpt-3.5-turbo'}]}

In [24]:
prompt = """
What does code do?
```
SELECT *
FROM people
WHERE name = "john"
```
"""

what_it_do = chat.completions.create(
   model="deimos/code-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(what_it_do.choices[0].message.content)

This is a SQL query. It returns every column for every row in the table named people where the column name equals "john".

Line-by-line:
- SELECT * — select all columns.
- FROM people — from the table called people.
- WHERE name = "john" — filter rows to those whose name equals "john".

Notes and gotchas:
- It produces a result set (zero or more rows); it does not modify the database.
- Standard SQL uses single quotes for string literals: WHERE name = 'john'. In some databases double quotes denote identifiers, and in others (e.g. default MySQL) double quotes may act like single quotes; behavior depends on the DBMS.
- Case-sensitivity depends on the column collation. For a case-insensitive match use e.g. WHERE LOWER(name) = 'john' or (in PostgreSQL) WHERE name ILIKE 'john'.
- For partial matches use LIKE (e.g. WHERE name LIKE '%john%').


In [25]:
what_it_do._deimos_metadata

{'router_used': 'code-lang-router',
 'selected_model': 'openai/gpt-5-mini',
 'original_model_field': 'openai/gpt-5-mini',
 'explain': [{'rule_type': 'CodeLanguageRule',
   'rule_name': 'code-lang-rule',
   'rule_trigger': 'sql',
   'decision': 'openai/gpt-5-mini'}]}

## NaturalLanguageRule

To route based on the language of the request (English, French, Spanish, etc), use a NaturalLanguageRule. This calls a small language model to detect the language. Specify language:model mapping using the two-letter ISO language code (`EN`, `FR`, `ES`).

In [27]:
NaturalLanguageRule(
    name = "nat-lang-rule",
    language_mappings = {
        "EN" : "openai/gpt-3.5-turbo",
        "ES" : "openai/gpt-5-mini",
    }
)

Router(
    name = "nat-lang-router",
    rules = ["deimos/rules/nat-lang-rule"]
)


Router('nat-lang-router', rules=['deimos/rules/nat-lang-rule'], default='gpt-3.5-turbo')

In [28]:
prompt = """
Tell me a joke about language.
"""

lang_joke = chat.completions.create(
   model="deimos/nat-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(lang_joke.choices[0].message.content)

Why did the grammar teacher go to jail? 
Because she kept committing homonyms!


In [29]:
lang_joke._deimos_metadata

{'router_used': 'nat-lang-router',
 'selected_model': 'openai/gpt-3.5-turbo',
 'original_model_field': 'openai/gpt-3.5-turbo',
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'EN',
   'decision': 'openai/gpt-3.5-turbo'}]}

In [30]:
prompt = """
Cuéntame un chiste sobre el lenguaje.
"""

lang_joke = chat.completions.create(
   model="deimos/nat-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(lang_joke.choices[0].message.content)

Un clásico corto:

Un lingüista entra en un bar. El camarero le pregunta: "¿Qué quiere tomar?" El lingüista responde: "Depende del contexto".

Y otro rápido:

—¿Por qué los sinónimos no pelean?  
—Porque siempre están de acuerdo en algo.


In [31]:
lang_joke._deimos_metadata

{'router_used': 'nat-lang-router',
 'selected_model': 'openai/gpt-5-mini',
 'original_model_field': 'openai/gpt-5-mini',
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'ES',
   'decision': 'openai/gpt-5-mini'}]}

## MessageLengthRule


The MessageLengthRule selects one of three models based on length (in tokens) of the user message. 

- short: below the `short_threshold`
- medium: between the `short_threshold` and the `long_threshold`
- long: above the `long_threshold`

In [32]:
MessageLengthRule(
    name = "msg-len",
    short_threshold = 50,
    long_threshold = 200,
    short_model = "openai/gpt-5-nano",
    medium_model = "openai/gpt-5-mini",
    long_model = "openai/gpt-5"
)

Router(
    name = "length-router",
    rules = ["deimos/rules/msg-len"],
)


Router('length-router', rules=['deimos/rules/msg-len'], default='gpt-3.5-turbo')

In [33]:
short_prompt = "What color is Mars?"

short_response= chat.completions.create(
   model="deimos/length-router",
   messages=[
       {"role": "user", "content": short_prompt}
   ],
)

print(short_response.choices[0].message.content)

Mars is generally described as reddish-orange, often called the Red Planet. This coloration comes from iron oxide (rust) on its surface and dust in the atmosphere. In some views it can look more orange-red, and its polar caps appear white.


In [34]:
short_response._deimos_metadata

{'router_used': 'length-router',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'explain': [{'rule_type': 'MessageLengthRule',
   'rule_name': 'msg-len',
   'rule_trigger': 'short_message_5_tokens',
   'decision': 'openai/gpt-5-nano'}]}

In [35]:
medium_prompt = "Imagine you’ve just been hired as the Chief Imagination Officer for a brand-new amusement park called DreamTopia. Your first assignment is to design the park’s most unusual and delightful attraction, something that no other park has ever seen before. It should combine at least two completely different ideas (for example: a roller coaster made of books, or a water slide that doubles as a musical instrument). Please describe the attraction in detail: how it looks, how it works, and what makes it magical or fun. End with a short tagline or slogan that could go on the park’s posters."

medium_response= chat.completions.create(
   model="deimos/length-router",
   messages=[
       {"role": "user", "content": medium_prompt}
   ],
)

print(medium_response.choices[0].message.content)

Name: The TokenTales SkyRail — “Stories that Fly”

Concept in one line
An aerial, slow-moving gondola ride that combines a cable-suspended scenic tram, a choose‑your‑own‑adventure storyteller’s toolkit (physical tokens), large-scale puppet theater and adaptive live music — so each trip becomes a unique, communal story told in motion.

How it looks
- The line and station feel like a giant open storybook: paper‑textured canopies, warm lantern light, and carved wooden shelves with glowing token bins.
- Gondola pods are cradle‑like “origami boats” of painted timber and glass (4–6 passengers each), suspended from a graceful cable that winds above a theatrical landscape of miniature worlds.
- Below and alongside the line are modular stage-sets — forests with mechanical trees, an “ocean” of reflective ribbons, clockwork cities, a mountain plateau with puppet giants, and hidden nooks for actors and animatronics to peek out.
- Lighting rigs, scent emitters, subtle wind and mist effects, project

In [36]:
medium_response._deimos_metadata

{'router_used': 'length-router',
 'selected_model': 'openai/gpt-5-mini',
 'original_model_field': 'openai/gpt-5-mini',
 'explain': [{'rule_type': 'MessageLengthRule',
   'rule_name': 'msg-len',
   'rule_trigger': 'medium_message_119_tokens',
   'decision': 'openai/gpt-5-mini'}]}

## ConversationContextRule

ConversationContextRule selects model based on the number of back-and-forth messages in a conversation, classifying a conversation as new, developing, or deep.

- new conversation: less than `new_threshold` number of messages
- developing conversation: between `new_threshold` and `deep_theshold` number of messages
- deep conversation: longer than `deep_threshold` number of messages

In [35]:
context_rule = ConversationContextRule(
    name = "conv-context-rule",
    new_threshold = 3,
    deep_threshold = 10,
    new_model = "openai/gpt-5-nano",
    developing_model = "openai/gpt-4o-mini",
    deep_model = "openai/gpt-4o"
)

Router(
    name = "context-router",
    rules = [context_rule]
)

Router('context-router', rules=[ConversationContextRule('conv-context-rule', 3, 10, 'openai/gpt-5-nano', 'openai/gpt-4o-mini', 'openai/gpt-4o')], default='gpt-3.5-turbo')

In [36]:
new_conversation= chat.completions.create(
   model="deimos/context-router",
   messages=[
       {"role": "user", "content": "Tell me the funnest fact about Deimos (the moon, not the god)."}
   ],
)

print(new_conversation.choices[0].message.content)

Fun fact: From Deimos, Mars would dominate the sky—Mars would appear about 16 degrees across (roughly 30 times the Moon’s apparent size from Earth). For contrast, Deimos itself is tiny (about 12 kilometers across) and potato-shaped, and it’s tidally locked to Mars, so the same face always points toward the planet.


In [37]:
new_conversation._deimos_metadata

{'router_used': 'context-router',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'explain': [{'rule_type': 'ConversationContextRule',
   'rule_name': 'conv-context-rule',
   'rule_trigger': 'new_conversation_1_messages_62_chars',
   'decision': 'openai/gpt-5-nano'}]}

In [38]:
conversation = [
    {
      "role": "user",
      "content": "Hi there! Can you help me come up with a fun name for a coffee shop?"
    },
    {
      "role": "assistant",
      "content": "Of course! How about 'Bean There, Done That'?"
    },
    {
      "role": "user",
      "content": "Haha, that’s clever. Can you give me a couple more options?"
    },
    {
      "role": "assistant",
      "content": "Sure! Some other ideas: 'Daily Grind Café' and 'Perk Up Coffeehouse.'"
    },
    {
      "role": "user",
      "content": "Nice! I like 'Perk Up.' Can you suggest a tagline to go with it?"
    },
    {
      "role": "assistant",
      "content": "How about: 'Perk Up — Where Every Cup Sparks Joy'?"
    },
    {
      "role": "user",
      "content": "Perfect, that’s exactly what I was looking for. Thanks!"
    }
  ]



In [39]:
developing_conversation= chat.completions.create(
   model="deimos/context-router",
   messages=conversation,
)

print(developing_conversation.choices[0].message.content)

You’re welcome! I’m glad you liked it. If you need any more help or ideas, feel free to ask. Enjoy your coffee shop!


In [40]:
developing_conversation._deimos_metadata

{'router_used': 'context-router',
 'selected_model': 'openai/gpt-4o-mini',
 'original_model_field': 'openai/gpt-4o-mini',
 'explain': [{'rule_type': 'ConversationContextRule',
   'rule_name': 'conv-context-rule',
   'rule_trigger': 'developing_conversation_7_messages_410_chars',
   'decision': 'openai/gpt-4o-mini'}]}

## Custom Rule Types

You can create your own custom Rule type by subclassing Rule and implementing `evaluate` with the following signature:

```
def evaluate(self, request_data: Dict[str, Any]) -> Decision:
    # return a Decision
```

A `Decision` has two arguments, a model str and a trigger str. The trigger str explains why the decision was made.

In [41]:
import random

class RandomRule(Rule):
    """Selects a model at random."""

    def __init__(self, name:str, models: list[str]):
        super().__init__(name)
        
        self.models = models

    def evaluate(self, _) -> Decision:
        model = random.choice(self.models)
        return Decision(model, "random_selection")
        

In [42]:
rand_rule = RandomRule(
    name = "rand-rule",
    models = [
        "openai/gpt-5-nano",
        "qwen/qwen-turbo",
        "x-ai/grok-3-mini"
    ]
)

Router(
    name = "random-router",
    rules = [rand_rule]
)

Router('random-router', rules=[RandomRule('rand-rule')], default='gpt-3.5-turbo')

In [43]:
for _ in range(5):

    prompt = "Tell me something about yourself."

    response= chat.completions.create(
       model="deimos/random-router",
       messages=[
           {"role": "user", "content": prompt}
       ],
    )
    
    print(response.choices[0].message.content)
    print()
    print(response._deimos_metadata)
    print("---")

Hello! I'm Qwen, a large-scale language model developed by Alibaba Group. I can help with a wide range of tasks, such as answering questions, creating text, writing stories, coding, and more. I'm here to assist you in any way I can. What would you like to ask or discuss? I'm looking forward to our conversation!

{'router_used': 'random-router', 'selected_model': 'qwen/qwen-turbo', 'original_model_field': 'qwen/qwen-turbo', 'explain': [{'rule_type': 'RandomRule', 'rule_name': 'rand-rule', 'rule_trigger': 'random_selection', 'decision': 'qwen/qwen-turbo'}]}
---
Sure, I'd be happy to! I'm Grok, an AI assistant created by xAI, with the goal of being super helpful, truthful, and a bit witty—like a mix of a know-it-all encyclopedia and the Hitchhiker's Guide to the Galaxy. I'm designed to answer questions, spark ideas, and tackle all sorts of topics without pulling punches, all while staying independent from any other companies' tech. What's on your mind next? 😊

{'router_used': 'random-rout

## Rule Chaining

Rules can call other rules instead of models. For example, you might want to first determine if code is present before determining which code language is being used.

In [73]:
code_lang_rule = CodeLanguageRule(
    name = "code-lang-rule",
    language_mappings = {
        "python" : "openai/gpt-3.5-turbo",
        "sql" : "openai/gpt-5-mini",
    }
)

code_or_not = CodeRule(
    name = "code-or-not-code",
    code = "deimos/rules/code-lang-rule",
    not_code = "openai/gpt-4o"
)



Router(
    name = "maybe_code_maybe_not",
    rules = [code_or_not]
)


Router('maybe_code_maybe_not', rules=[CodeRule('code-or-not-code', 'deimos/rules/code-lang-rule', 'openai/gpt-4o')], default='gpt-3.5-turbo')

In [74]:
not_code_prompt = "Tell me a joke."

not_code_response= chat.completions.create(
   model="deimos/maybe_code_maybe_not",
   messages=[
       {"role": "user", "content": not_code_prompt}
   ],
)

print(not_code_response.choices[0].message.content)

Why don't skeletons fight each other?

They don't have the guts.


In [75]:
not_code_response._deimos_metadata

{'router_used': 'maybe_code_maybe_not',
 'selected_model': 'openai/gpt-4o',
 'original_model_field': 'openai/gpt-4o',
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'no_code_detected',
   'decision': 'openai/gpt-4o'}]}

In [77]:
code_prompt = """
def f(x):
    return x**2
"""

code_response = chat.completions.create(
   model="deimos/maybe_code_maybe_not",
   messages=[
       {"role": "user", "content": code_prompt}
   ],
)

print(code_response.choices[0].message.content)

This is a simple function named f that takes a parameter x and returns the square of x.


In [78]:
code_response._deimos_metadata

{'router_used': 'maybe_code_maybe_not',
 'selected_model': 'openai/gpt-3.5-turbo',
 'original_model_field': 'openai/gpt-3.5-turbo',
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'code_detected',
   'decision': 'continue'},
  {'rule_type': 'CodeLanguageRule',
   'rule_name': 'code-lang-rule',
   'rule_trigger': 'python',
   'decision': 'openai/gpt-3.5-turbo'}]}

## Rule Fallthrough

You can also list more than one rule in the router. In this case, if the first rule does not match any defined case, the next rule will be used, and so on until a rule that returns a model is found. You can also define a default model on a router, which is used if no rule returns a model.

In [79]:
nat_lang_rule = NaturalLanguageRule(
    name = "nat-lang-rule",
    language_mappings = {
        "FR" : "openai/gpt-3.5-turbo",
        "ES" : "openai/gpt-5-mini",
    }
)

auto_task = AutoTaskRule(
    name = "auto-task-rule",
    triggers = {
        "writing code" : "openai/gpt-5",
        "medical advice" : "openai/gpt-5-mini",
        "haiku composition" : "openai/gpt-5-nano"
    },
)

Router(
    name = "haiku-but-not-limericks",
    rules = [nat_lang_rule, auto_task],
    default = "openai/gpt-4o"
)

Router('haiku-but-not-limericks', rules=[NaturalLanguageRule('nat-lang-rule', {'FR': 'openai/gpt-3.5-turbo', 'ES': 'openai/gpt-5-mini'}, default=None, llm_model='openai/gpt-4o-mini'), AutoTaskRule('auto-task-rule', {'writing code': 'openai/gpt-5', 'medical advice': 'openai/gpt-5-mini', 'haiku composition': 'openai/gpt-5-nano'}, default=None, llm_model='openai/gpt-5-nano')], default='openai/gpt-4o')

In [80]:
haiku_request = "Write me a short poem with three lines and 17 syllables on Deimos (the god, not the moon)."

haiku_response = chat.completions.create(
   model="deimos/haiku-but-not-limericks",
   messages=[
       {"role": "user", "content": haiku_request}
   ],
)

print(haiku_response.choices[0].message.content)

Deimos, god of dread
From war's shadow, he moves near.
Fear trails in his wake.


In [81]:
haiku_response._deimos_metadata

{'router_used': 'haiku-but-not-limericks',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'no_language_detected',
   'decision': 'no_match'},
  {'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'haiku composition',
   'decision': 'openai/gpt-5-nano'}]}

In [82]:
limerick_request = "Write a funny five line poem, with AABBA rhyme scheme and a sing-songy meter, about any computer science subject."

limerick_response = chat.completions.create(
   model="deimos/haiku-but-not-limericks",
   messages=[
       {"role": "user", "content": limerick_request}
   ],
)

print(limerick_response.choices[0].message.content)

A coder set out on a quest,  
To optimize code for a test,  
But the loops nested deep,  
Made his sanity leap,  
Now his brain’s out for garbage collect!


In [83]:
limerick_response._deimos_metadata

{'router_used': 'haiku-but-not-limericks',
 'selected_model': 'openai/gpt-4o',
 'original_model_field': 'openai/gpt-4o',
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'no_language_detected',
   'decision': 'no_match'},
  {'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'None',
   'decision': 'no_match'},
  {'rule_type': 'default',
   'rule_name': 'default',
   'rule_trigger': 'None',
   'decision': 'openai/gpt-4o'}]}